In [7]:
import langchain
import dotenv
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.dialects.postgresql import insert
import os

dotenv.load_dotenv()

embedding = langchain.embeddings.OpenAIEmbeddings(
    model='text-embedding-ada-002',
)

DATABASE_URL = os.environ.get('DATABASE_URL', None)
# Connect to the database
connection = create_engine(DATABASE_URL)
# Reflect the database schema
schema = automap_base()
schema.prepare(connection, reflect=True)
session = Session(connection)


/var/folders/rl/l_q2y9td2rx0jf4kf0hfy0qm0000gn/T/ipykernel_10323/1435692714.py:20: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  schema.prepare(connection, reflect=True)
/var/folders/rl/l_q2y9td2rx0jf4kf0hfy0qm0000gn/T/ipykernel_10323/1435692714.py:20: SAWarning: Did not recognize type 'vector' of column 'embedding'
  schema.prepare(connection, reflect=True)


In [8]:
with open('cypher_queries.txt','r') as f:
    queries = f.readlines()

In [9]:
nodes = []
for query in queries:
    if "CREATE" == query[:6]:
        nodes += [query.split('"')[1]]


In [10]:
embeddings_list = embedding.embed_documents(nodes)

In [11]:
records = []
for i in range(len(nodes)):
    records += [{"content":nodes[i],"embedding":embeddings_list[i]}]
    

In [12]:
nodes_embeddings_table = schema.classes.nodes_embeddings
insert_data = insert(nodes_embeddings_table).values(records)
insert_data = insert_data.on_conflict_do_nothing()
try:
    session.execute(insert_data)
    session.commit()

except Exception as e:
    session.rollback()
    raise e